In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def ptaa_updated():
    # Inisialisasi dictionary dengan menambahkan kunci baru "abstrak_inggris"
    data = {
        "penulis": [],
        "judul": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "abstrak": [],
        "abstrak_inggris": []  # Kolom baru untuk abstrak Bahasa Inggris
    }

    # Looping tetap sama
    for i in range(1, 4):
        url = f"https://pta.trunojoyo.ac.id/c_search/byprod/10/{i}"
        try:
            r = requests.get(url)
            r.raise_for_status()  # Memeriksa jika ada error pada request
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            for jurnal in jurnals:
                jurnal_url = jurnal.select_one('a.gray.button')['href']
                response = requests.get(jurnal_url)
                response.raise_for_status()
                soup1 = BeautifulSoup(response.content, "html.parser")

                isi = soup1.select_one('div#content_journal')

                if not isi:
                    continue

                # Mengambil data yang sudah ada
                judul = isi.select_one('a.title').text.strip()
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()

                # --- MODIFIKASI UNTUK ABSTRAK ---
                # Memilih semua paragraf dengan 'align="justify"'
                abstract_paragraphs = isi.select('p[align="justify"]')

                # Paragraf pertama adalah abstrak Bahasa Indonesia
                abstrak_indonesia = abstract_paragraphs[0].text.strip() if len(abstract_paragraphs) > 0 else "Tidak ada abstrak"

                # Paragraf kedua adalah abstrak Bahasa Inggris
                abstrak_inggris = abstract_paragraphs[1].text.strip() if len(abstract_paragraphs) > 1 else "No abstract available"
                # --- AKHIR DARI MODIFIKASI ---

                # Menambahkan semua data ke dictionary
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["abstrak"].append(abstrak_indonesia)
                data["abstrak_inggris"].append(abstrak_inggris) # Menambahkan data baru

        except requests.exceptions.RequestException as e:
            print(f"Terjadi error: {e}")
            continue

    df = pd.DataFrame(data)
    df.to_csv("pta_updated.csv", index=False)

    return df

# Untuk menjalankan fungsi
updated_dataframe = ptaa_updated()
print(updated_dataframe.head())

C:\laragon\bin\python\python-3.10\lib\site-packages\soupsieve\css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


                 penulis                                              judul  \
0     A.Ubaidillah S.Kom  PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...   
1    M. Basith Ardianto,  APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...   
2  Akhmad Suyandi, S.Kom  RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...   
3        Heri Supriyanto  SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...   
4   Septian Rahman Hakim  SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...   

              pembimbing_pertama           pembimbing_kedua  \
0               Budi Setyono M.T               Hermawan S.T   
1          Drs. Budi Soesilo, MT              Koko Joni, ST   
2         Drs. Budi Soesilo, M.T           Hermawan, ST, MT   
3           Mulaab, S.Si., M.Kom  Firli Irhamni, ST., M.Kom   
4  Arik Kurniawati, S.Kom., M.T.       Haryanto, S.T., M.T.   

                                             abstrak  \
0  Sistem  informasi  akademik  (SIAKAD) merupaka...   
1  Berjalannya koneksi jaringan ko

In [2]:
ptaa_updated()

,penulis,judul,pembimbing_pertama,pembimbing_kedua,abstrak,abstrak_inggris
0,A.Ubaidillah S.Kom,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...,Academic information systems (SIAKAD) is an in...
1,"M. Basith Ardianto,",APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...,-
2,"Akhmad Suyandi, S.Kom",RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,"Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...,Web server is a server software functioning to...
3,Heri Supriyanto,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...,Scheduling courses in universities is a ...
4,Septian Rahman Hakim,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...,As the development of technology existing in t...
5,Adi Chandra Laksono,Gerak Pekerja Pada Game Real Time Strategy Men...,"Kurniawan Eka P, S.Kom., Msc","Arik Kurniawati, S.Kom., M.T.",Gerak pekerja ada pada game yang memiliki genr...,Workers' movements in the game that has no gen...
6,NURRACHMAT,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,"Arik Kurniawati, S.Kom., M.T.","Kurniawan Eka Permana, S.Kom., MSc.","Perkembangan game yang semakin pesat, memberik...","The rapid development of the game, providing a..."
7,Muhammad Choirur Rozi,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,"Dr. Arif Muntasa, S.Si.,M.T","Fitri Damayanti, S.Kom.,M.kom",Sistem pengenalan wajah adalah suatu sistem un...,Face recognition system is a system to recogni...
8,M Khoiril Anwar,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,"Cucun Very Angkoso, S.T., M.T.","Arik Kurniawati S. Kom., M.T.",Teknologi mobile game beroperating system open...,Mobile technology is an open source game devel...
9,MALIKUL HAMZAH,Perancangan Sistem Informasi Badan Kepegawaian...,"Moch. Kautsar Sophan, S.Kom., M.MT.","Yeni Kustiyaningsih, S.Kom., M.Kom.",Kantor Badan Kepegawaian kota Bangkalan adalah...,Office of Personnel Agency is Bangkalan city g...


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

def get_fakultas_prodi_list():
    """
    Fungsi untuk mengambil daftar semua Fakultas dan Prodi di dalamnya.
    (Fungsi ini tidak berubah dari sebelumnya)
    """
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav:
            print("Sidebar navigasi tidak ditemukan.")
            return []

        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue

            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar fakultas dan prodi: {e}")
        
    return prodi_list

def scrape_all():
    """
    Fungsi utama untuk scraping data, dimodifikasi untuk mengambil
    maksimal 4 data per prodi.
    """
    daftar_prodi_lengkap = get_fakultas_prodi_list()
    if not daftar_prodi_lengkap:
        print("Tidak ada prodi yang bisa di-scrape. Program berhenti.")
        return

    data = {
        "nama_fakultas": [], "id_prodi": [], "nama_prodi": [],
        "penulis": [], "judul": [], "pembimbing_pertama": [],
        "pembimbing_kedua": [], "abstrak": [], "abstrak_inggris": []
    }

    for prodi in daftar_prodi_lengkap:
        prodi_id = prodi["id_prodi"]
        nama_prodi = prodi["nama_prodi"]
        nama_fakultas = prodi["nama_fakultas"]
        page = 1
        
        # --- PERUBAHAN DIMULAI DI SINI ---
        
        # 1. Tambahkan counter untuk menghitung jurnal yang sudah diambil per prodi
        jurnal_diambil_count = 0
        
        # 2. Tambahkan 'flag' untuk menandai jika batas sudah tercapai
        limit_tercapai = False
        
        # --- AKHIR PERUBAHAN ---

        while True:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi_id}/{page}"
            try:
                r = requests.get(url)
                r.raise_for_status()
                soup = BeautifulSoup(r.content, "html.parser")
                jurnals = soup.select('li[data-cat="#luxury"]')

                if not jurnals:
                    break

                for jurnal in jurnals:
                    # --- PERUBAHAN DIMULAI DI SINI ---
                    
                    # 3. Cek apakah counter sudah mencapai 4, jika ya, hentikan loop
                    if jurnal_diambil_count >= 1:
                        limit_tercapai = True # Set flag menjadi True
                        break # Hentikan loop 'for jurnal in jurnals'
                    
                    # --- AKHIR PERUBAHAN ---

                    # Proses scraping detail (tetap sama)
                    jurnal_url = jurnal.select_one('a.gray.button')['href']
                    response = requests.get(jurnal_url)
                    response.raise_for_status()
                    isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
                    if not isi: continue

                    # (Kode untuk mengambil judul, penulis, dll. tidak diubah)
                    judul = isi.select_one('a.title').text.strip()
                    penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
                    pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
                    pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
                    # --- PERUBAHAN PADA EKSTRAKSI ABSTRAK ---
                    abstract_paragraphs = isi.select('p[align="justify"]')
                    
                    # Ambil teks mentah dari abstrak Bahasa Indonesia
                    text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else "Tidak ada abstrak"
                    # 2. BERSIHKAN TEKS MENTAH
                    abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()

                    # Ambil teks mentah dari abstrak Bahasa Inggris
                    text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else "No abstract available"
                    # 2. BERSIHKAN TEKS MENTAH
                    abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()
                    # --- AKHIR PERUBAHAN ---

                    # Menambahkan semua data ke dictionary
                    data["nama_fakultas"].append(nama_fakultas)
                    data["id_prodi"].append(prodi_id)
                    data["nama_prodi"].append(nama_prodi)
                    data["penulis"].append(penulis)
                    data["judul"].append(judul)
                    data["pembimbing_pertama"].append(pembimbing_pertama)
                    data["pembimbing_kedua"].append(pembimbing_kedua)
                    data["abstrak"].append(abstrak_indonesia)
                    data["abstrak_inggris"].append(abstrak_inggris)
                    
                    # --- PERUBAHAN DIMULAI DI SINI ---
                    
                    # 4. Tambah nilai counter setiap kali satu jurnal berhasil diambil
                    jurnal_diambil_count += 1
                    
                    # --- AKHIR PERUBAHAN ---
                
                # --- PERUBAHAN DIMULAI DI SINI ---
                
                # 5. Cek flag, jika True, hentikan juga loop halaman (while)
                if limit_tercapai:
                    break
                    
                # --- AKHIR PERUBAHAN ---
                
                page += 1
                time.sleep(1)

            except requests.exceptions.RequestException as e:
                print(f"Error pada Prodi {nama_prodi} (ID: {prodi_id}) halaman {page}: {e}")
                break
        
        print(f"✔️ Selesai: {nama_fakultas} - {nama_prodi} (ID: {prodi_id}) | Berhasil mengambil {jurnal_diambil_count} jurnal.")

    df = pd.DataFrame(data)
    df.to_csv("pta_4_data_per_prodi.csv", index=False)
    print("\n✅ Proses scraping selesai. Data disimpan ke 'pta_4_data_per_prodi.csv'")
    
    return df

# Untuk menjalankan seluruh proses scraping
# df_final = scrape_all()
# print(df_final)

In [4]:
scrape_all()

✔️ Selesai: Hukum - Ilmu Hukum (ID: 1) | Berhasil mengambil 1 jurnal.


✔️ Selesai: Hukum - Magister Ilmu Hukum (ID: 24) | Berhasil mengambil 1 jurnal.


✔️ Selesai: Pertanian - Teknologi Industri Pertanian (ID: 2) | Berhasil mengambil 1 jurnal.


✔️ Selesai: Pertanian - Agribisnis (ID: 3) | Berhasil mengambil 1 jurnal.


✔️ Selesai: Pertanian - Agroteknologi (ID: 4) | Berhasil mengambil 1 jurnal.


...

✔️ Selesai: Ekonomi Dan Bisnis - Magister Ilmu Ekonomi (ID: 36) | Berhasil mengambil 1 jurnal.


✔️ Selesai: Ekonomi Dan Bisnis - Doktor Ilmu Manajemen (ID: 41) | Berhasil mengambil 0 jurnal.


✔️ Selesai: Teknik - Teknik Industri (ID: 9) | Berhasil mengambil 1 jurnal.


✔️ Selesai: Teknik - Teknik Informatika (ID: 10) | Berhasil mengambil 1 jurnal.


KeyboardInterrupt: 

In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures

# MAKSIMAL PEKERJA SIMULTAN (BISA DISESUAIKAN)
MAX_WORKERS = 10

def get_fakultas_prodi_list():
    # Fungsi ini tidak berubah, tetap sama seperti sebelumnya
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav: return []
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar prodi: {e}")
    return prodi_list

def scrape_jurnal_detail(jurnal_url):
    """
    FUNGSI PEKERJA: Hanya bertugas men-scrape detail SATU jurnal.
    Fungsi inilah yang akan dijalankan oleh setiap thread.
    """
    try:
        response = requests.get(jurnal_url, timeout=15)
        response.raise_for_status()
        isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
        if not isi: return None

        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
        
        abstract_paragraphs = isi.select('p[align="justify"]')
        text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else ""
        abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()
        text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else ""
        abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()

        return {
            "penulis": penulis, "judul": judul, "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua, "abstrak": abstrak_indonesia,
            "abstrak_inggris": abstrak_inggris
        }
    except requests.exceptions.RequestException:
        return None # Return None jika gagal scrape satu jurnal

def scrape_prodi(prodi):
    """
    FUNGSI MANAJER: Bertugas mencari URL jurnal untuk SATU prodi,
    lalu menyuruh 'pekerja' untuk men-scrape detailnya.
    """
    jurnal_urls = []
    page = 1
    while len(jurnal_urls) < 4:
        try:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi['id_prodi']}/{page}"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals_on_page = soup.select('li[data-cat="#luxury"] a.gray.button')

            if not jurnals_on_page:
                break # Hentikan jika tidak ada jurnal lagi di halaman

            for a_tag in jurnals_on_page:
                if len(jurnal_urls) < 4:
                    jurnal_urls.append(a_tag['href'])
                else:
                    break
            page += 1
        except requests.exceptions.RequestException:
            break # Hentikan jika ada error jaringan

    if not jurnal_urls:
        return []

    all_jurnal_data = []
    # Jalankan thread pool untuk men-scrape semua URL detail yang sudah terkumpul
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        hasil_scrape = executor.map(scrape_jurnal_detail, jurnal_urls)
        for hasil in hasil_scrape:
            if hasil: # Pastikan hasilnya tidak None
                # Tambahkan info prodi dan fakultas ke hasil scrape
                hasil['id_prodi'] = prodi['id_prodi']
                hasil['nama_prodi'] = prodi['nama_prodi']
                hasil['nama_fakultas'] = prodi['nama_fakultas']
                all_jurnal_data.append(hasil)
    
    print(f"✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Berhasil mengambil {len(all_jurnal_data)} jurnal.")
    return all_jurnal_data


def main():
    """Fungsi utama untuk mengorkestrasi seluruh proses."""
    start_time = time.time()
    
    daftar_prodi = get_fakultas_prodi_list()
    if not daftar_prodi: return

    final_results = []
    # Kita bahkan bisa menjalankan scraping antar prodi secara konkuren!
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Jalankan fungsi scrape_prodi untuk setiap item di daftar_prodi
        results_per_prodi = executor.map(scrape_prodi, daftar_prodi)
        for list_jurnal in results_per_prodi:
            final_results.extend(list_jurnal)

    df = pd.DataFrame(final_results)
    # Atur urutan kolom agar lebih rapi
    if not df.empty:
        df = df[['nama_fakultas', 'id_prodi', 'nama_prodi', 'judul', 'penulis', 'pembimbing_pertama', 'pembimbing_kedua', 'abstrak', 'abstrak_inggris']]
    
    df.to_csv("pta_final_concurrent.csv", index=False)
    
    end_time = time.time()
    print("\n✅ Proses scraping selesai.")
    print(f"Total data yang berhasil diambil: {len(df)} jurnal.")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")

if __name__ == "__main__":
    main()

✔️ Selesai: Pertanian - Agroteknologi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Hukum - Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Ilmu Kelautan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Teknologi Industri Pertanian | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Manajemen Sumberdaya Perairan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Ekonomi Pembangunan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Manajemen | Berhasil mengambil 4 jurnal.
✔️ Selesai: Hukum - Magister Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Agribisnis | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Magister Pengelolaan Sumber Daya Alam | Berhasil mengambil 3 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Akuntansi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - D3 Akuntansi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Magister Manajemen | Berhasil mengambil 4 jurnal.
✔️ Selesa

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures

# MAKSIMAL PEKERJA SIMULTAN (BISA DISESUAIKAN)
MAX_WORKERS = 10

def get_fakultas_prodi_list():
    # Fungsi ini tidak berubah, tetap sama seperti sebelumnya
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav: return []
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar prodi: {e}")
    return prodi_list

def scrape_jurnal_detail(jurnal_url):
    # Fungsi ini tidak berubah
    try:
        response = requests.get(jurnal_url, timeout=15)
        response.raise_for_status()
        isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
        if not isi: return None

        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
        
        abstract_paragraphs = isi.select('p[align="justify"]')
        text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else ""
        abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()
        text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else ""
        abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()

        return {
            "penulis": penulis, "judul": judul, "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua, "abstrak": abstrak_indonesia,
            "abstrak_inggris": abstrak_inggris
        }
    except requests.exceptions.RequestException:
        return None

def scrape_prodi(prodi):
    # Fungsi ini tidak berubah
    jurnal_urls = []
    page = 1
    while len(jurnal_urls) < 4:
        try:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi['id_prodi']}/{page}"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals_on_page = soup.select('li[data-cat="#luxury"] a.gray.button')
            if not jurnals_on_page: break
            for a_tag in jurnals_on_page:
                if len(jurnal_urls) < 4:
                    jurnal_urls.append(a_tag['href'])
                else: break
            page += 1
        except requests.exceptions.RequestException:
            break

    if not jurnal_urls: return []

    all_jurnal_data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        hasil_scrape = executor.map(scrape_jurnal_detail, jurnal_urls)
        for hasil in hasil_scrape:
            if hasil:
                hasil['id_prodi'] = prodi['id_prodi']
                hasil['nama_prodi'] = prodi['nama_prodi']
                hasil['nama_fakultas'] = prodi['nama_fakultas']
                all_jurnal_data.append(hasil)
    
    print(f"✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Berhasil mengambil {len(all_jurnal_data)} jurnal.")
    return all_jurnal_data

def main():
    start_time = time.time()
    
    daftar_prodi = get_fakultas_prodi_list()
    if not daftar_prodi: return pd.DataFrame() # Return DataFrame kosong jika gagal

    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results_per_prodi = executor.map(scrape_prodi, daftar_prodi)
        for list_jurnal in results_per_prodi:
            final_results.extend(list_jurnal)

    df = pd.DataFrame(final_results)
    if not df.empty:
        df = df[['nama_fakultas', 'id_prodi', 'nama_prodi', 'judul', 'penulis', 'pembimbing_pertama', 'pembimbing_kedua', 'abstrak', 'abstrak_inggris']]
    
    df.to_csv("pta_final_concurrent.csv", index=False)
    
    end_time = time.time()
    print("\n✅ Proses scraping selesai.")
    print(f"Total data yang berhasil diambil: {len(df)} jurnal.")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    # --- PERUBAHAN DI SINI ---
    # 1. Kembalikan DataFrame yang sudah jadi
    return df

# --- DAN PERUBAHAN DI SINI ---
if __name__ == "__main__":
    # 2. Panggil fungsi dan simpan hasilnya ke variabel
    df_hasil = main()
    
    # 3. Letakkan variabel di baris terakhir untuk menampilkannya di Colab/Jupyter
    df_hasil

✔️ Selesai: Ekonomi Dan Bisnis - Ekonomi Pembangunan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Agroteknologi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Hukum - Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Ilmu Kelautan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Agribisnis | Berhasil mengambil 4 jurnal.
✔️ Selesai: Hukum - Magister Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Teknologi Industri Pertanian | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Manajemen | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Manajemen Sumberdaya Perairan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Magister Pengelolaan Sumber Daya Alam | Berhasil mengambil 3 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Akuntansi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - D3 Akuntansi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Magister Manajemen | Berhasil mengambil 4 jurnal.
✔️ Selesa

In [16]:
main()

✔️ Selesai: Hukum - Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Hukum - Magister Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Ekonomi Pembangunan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Manajemen Sumberdaya Perairan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Manajemen | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Ilmu Kelautan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Agroteknologi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Teknologi Industri Pertanian | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Agribisnis | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Magister Pengelolaan Sumber Daya Alam | Berhasil mengambil 3 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Magister Ilmu Ekonomi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Teknik - Teknik Industri | Berhasil mengambil 4 jurnal.
✔️ Selesai: Teknik - Teknik Informatika | Berhasil mengambil 4 jurnal.
✔️ Selesai: Teknik

,nama_fakultas,id_prodi,nama_prodi,judul,penulis,pembimbing_pertama,pembimbing_kedua,abstrak,abstrak_inggris
0,Hukum,1,Ilmu Hukum,Implementasi Fungsi Legislasi Dewan Perwakilan...,Dyah Ayu Citra Seza,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",ABSTRAK Implementasi Fungsi Legislasi DPRD Kab...,ABSTRACT Implementation of Legislation Parliam...
1,Hukum,1,Ilmu Hukum,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Maulina Nurlaily,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...
2,Hukum,1,Ilmu Hukum,Analisis Terhadap Kekosongan Hukum dalam Penga...,Moh. Samsul Hidayat,"Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi..."
3,Hukum,1,Ilmu Hukum,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,TOMMY ADITYA PARLINDUNGAN MARBUN,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...
4,Hukum,24,Magister Ilmu Hukum,PERTANGGUNG JAWABAN HUKUM PEJABAT PEMBUAT KOMI...,MOCHAMAD SAICHU,"Dr. Nunuk Nuswardani S.H., M.H","Dr. Deni SB Yuherawan S.H., Msi",Korupsi tergolong ke dalam kejahatan yang luar...,Korupsi tergolong ke dalam kejahatan yang luar...
...,...,...,...,...,...,...,...,...,...
98,Ilmu Pendidikan,29,Pendidikan Ipa,PENGEMBANGAN LEMBAR KERJA SISWABERORIENTASI \n...,Aden Toyyib,"Yunin Hidayati, S.Si., M.Si","Irsad Rosidi, S.Pd., M.Pd",Penelitian ini merupakan penelitian pengembang...,The research aims to know validity of socio sc...
99,Ilmu Pendidikan,30,Pgpaud,Hubungan Antara Aktivitas Fisik Dengan Kedisip...,Indri Wulan Suryani,"Titin Faridatun Nisa', M.Pd","Yulias Wulani Fajar, M.Pd",Pembelajaran pada anak usia dini dilakukan mel...,ABSTRAK Learning in early childhood has been d...
100,Ilmu Pendidikan,30,Pgpaud,Efektivitas Penanaman Nilai Agama untuk Membe...,Endang Ratnasari,"Titin Faridatun Nisa', M.Pd","Yulias Wulani Fajar, M.Pd",Penanaman nilai agama merupakan salah satu pem...,Applying religion value is one of the more dom...
101,Ilmu Pendidikan,30,Pgpaud,PENGARUH METODE DEMONSTRASI TERHADAP PERILAKU ...,Lischa Dwi Christin Niya Ningrum,"Muhammad Busyro Karim, S.Ag., M.Si.","Dewi Mayangsari, M.Psi",Kesehatan mempunyai pengaruh terhadap pencapai...,Health has an influence on the achievement of ...


In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures

# MAKSIMAL PEKERJA SIMULTAN (BISA DISESUAIKAN)
MAX_WORKERS = 10

def get_fakultas_prodi_list():
    # Fungsi ini tidak berubah
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav: return []
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar prodi: {e}")
    return prodi_list

def scrape_jurnal_detail(jurnal_url):
    # Fungsi ini tidak berubah
    try:
        response = requests.get(jurnal_url, timeout=15)
        response.raise_for_status()
        isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
        if not isi: return None
        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
        abstract_paragraphs = isi.select('p[align="justify"]')
        text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else ""
        abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()
        text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else ""
        abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()
        return {
            "penulis": penulis, "judul": judul, "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua, "abstrak": abstrak_indonesia,
            "abstrak_inggris": abstrak_inggris
        }
    except requests.exceptions.RequestException:
        return None

# --- PERUBAHAN UTAMA ADA DI FUNGSI BERIKUT ---
def scrape_prodi(prodi):
    """
    FUNGSI MANAJER: Sekarang akan mengembalikan baris data kosong
    jika tidak ada jurnal yang ditemukan.
    """
    jurnal_urls = []
    page = 1
    while len(jurnal_urls) < 4:
        try:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi['id_prodi']}/{page}"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals_on_page = soup.select('li[data-cat="#luxury"] a.gray.button')
            if not jurnals_on_page: break
            for a_tag in jurnals_on_page:
                if len(jurnal_urls) < 4:
                    jurnal_urls.append(a_tag['href'])
                else: break
            page += 1
        except requests.exceptions.RequestException:
            break

    # --- BAGIAN YANG DIMODIFIKASI ---
    # Jika setelah mencari, tidak ada URL jurnal yang ditemukan
    if not jurnal_urls:
        print(f"✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Ditemukan 0 jurnal.")
        # Buat satu baris data placeholder
        empty_row = {
            'id_prodi': prodi['id_prodi'],
            'nama_prodi': prodi['nama_prodi'],
            'nama_fakultas': prodi['nama_fakultas'],
            'judul': 'Tidak ada jurnal',
            'penulis': None,
            'pembimbing_pertama': None,
            'pembimbing_kedua': None,
            'abstrak': None,
            'abstrak_inggris': None
        }
        return [empty_row] # Kembalikan sebagai list berisi satu dictionary
    # --- AKHIR MODIFIKASI ---

    # Jika ada URL, proses berjalan seperti biasa
    all_jurnal_data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        hasil_scrape = executor.map(scrape_jurnal_detail, jurnal_urls)
        for hasil in hasil_scrape:
            if hasil:
                hasil['id_prodi'] = prodi['id_prodi']
                hasil['nama_prodi'] = prodi['nama_prodi']
                hasil['nama_fakultas'] = prodi['nama_fakultas']
                all_jurnal_data.append(hasil)
    
    print(f"✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Berhasil mengambil {len(all_jurnal_data)} jurnal.")
    return all_jurnal_data

def main():
    # Fungsi ini tidak berubah
    start_time = time.time()
    daftar_prodi = get_fakultas_prodi_list()
    if not daftar_prodi: return pd.DataFrame()

    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results_per_prodi = executor.map(scrape_prodi, daftar_prodi)
        for list_jurnal in results_per_prodi:
            final_results.extend(list_jurnal)

    df = pd.DataFrame(final_results)
    if not df.empty:
        df = df[['nama_fakultas', 'id_prodi', 'nama_prodi', 'judul', 'penulis', 'pembimbing_pertama', 'pembimbing_kedua', 'abstrak', 'abstrak_inggris']]
    
    df.to_csv("pta_final_concurrent_lengkap.csv", index=False)
    
    end_time = time.time()
    print("\n✅ Proses scraping selesai.")
    print(f"Total baris data yang dihasilkan: {len(df)}.")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    return df

# if __name__ == "__main__":
#     df_hasil = main()
#     df_hasil

In [18]:
main()

✔️ Selesai: Pertanian - Agroteknologi | Berhasil mengambil 4 jurnal.
✔️ Selesai: Hukum - Magister Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Hukum - Ilmu Hukum | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Teknologi Industri Pertanian | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Manajemen Sumberdaya Perairan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Agribisnis | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Ekonomi Pembangunan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Manajemen | Berhasil mengambil 4 jurnal.
✔️ Selesai: Pertanian - Ilmu Kelautan | Berhasil mengambil 4 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Akuntansi | Ditemukan 0 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - D3 Akuntansi | Ditemukan 0 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Magister Manajemen | Ditemukan 0 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - Magister Akuntansi | Ditemukan 0 jurnal.
✔️ Selesai: Ekonomi Dan Bisnis - D3 Enterpreneurship | 

,nama_fakultas,id_prodi,nama_prodi,judul,penulis,pembimbing_pertama,pembimbing_kedua,abstrak,abstrak_inggris
0,Hukum,1,Ilmu Hukum,Implementasi Fungsi Legislasi Dewan Perwakilan...,Dyah Ayu Citra Seza,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",ABSTRAK Implementasi Fungsi Legislasi DPRD Kab...,ABSTRACT Implementation of Legislation Parliam...
1,Hukum,1,Ilmu Hukum,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Maulina Nurlaily,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...
2,Hukum,1,Ilmu Hukum,Analisis Terhadap Kekosongan Hukum dalam Penga...,Moh. Samsul Hidayat,"Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi..."
3,Hukum,1,Ilmu Hukum,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,TOMMY ADITYA PARLINDUNGAN MARBUN,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...
4,Hukum,24,Magister Ilmu Hukum,PERTANGGUNG JAWABAN HUKUM PEJABAT PEMBUAT KOMI...,MOCHAMAD SAICHU,"Dr. Nunuk Nuswardani S.H., M.H","Dr. Deni SB Yuherawan S.H., Msi",Korupsi tergolong ke dalam kejahatan yang luar...,Korupsi tergolong ke dalam kejahatan yang luar...
...,...,...,...,...,...,...,...,...,...
86,Ilmu Pendidikan,30,Pgpaud,Pengaruh Strategi Meaningful Instructional Des...,Siti Herlinah Wifroh,"Yulias Wulani Fajar, M.Pd","Titin Faridatun Nisa', S.Pd., M.Pd",Tujuan penelitian ini untuk mengetahui pengaru...,The purpose of this study is to reveal the inf...
87,Ilmu Pendidikan,38,Pendidikan Profesi Guru,Tidak ada jurnal,None,None,None,None,None
88,Mbkm,99,Mbkm,Tidak ada jurnal,None,None,None,None,None
89,Pascasarjana,39,Magister Pendidikan Dasar,Tidak ada jurnal,None,None,None,None,None


In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures

# MAKSIMAL PEKERJA SIMULTAN (BISA DISESUAIKAN)
MAX_WORKERS = 10

def get_fakultas_prodi_list():
    # ... (fungsi ini tidak berubah)
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav: return []
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar prodi: {e}")
    return prodi_list

def scrape_jurnal_detail(jurnal_url):
    # ... (fungsi ini tidak berubah)
    try:
        response = requests.get(jurnal_url, timeout=15)
        response.raise_for_status()
        isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
        if not isi: return None
        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
        abstract_paragraphs = isi.select('p[align="justify"]')
        text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else ""
        abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()
        text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else ""
        abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()
        return {
            "penulis": penulis, "judul": judul, "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua, "abstrak": abstrak_indonesia,
            "abstrak_inggris": abstrak_inggris
        }
    except requests.exceptions.RequestException:
        return None

def scrape_prodi(prodi):
    # ... (fungsi ini tidak berubah, kecuali pada bagian print)
    jurnal_urls = []
    page = 1
    while len(jurnal_urls) < 4:
        try:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi['id_prodi']}/{page}"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals_on_page = soup.select('li[data-cat="#luxury"] a.gray.button')
            if not jurnals_on_page: break
            for a_tag in jurnals_on_page:
                if len(jurnal_urls) < 4:
                    jurnal_urls.append(a_tag['href'])
                else: break
            page += 1
        except requests.exceptions.RequestException:
            break

    if not jurnal_urls:
        # Tambahkan \n di awal print agar lebih rapi
        print(f"\n✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Ditemukan 0 jurnal.")
        return [{
            'id_prodi': prodi['id_prodi'], 'nama_prodi': prodi['nama_prodi'],
            'nama_fakultas': prodi['nama_fakultas'], 'judul': 'Tidak ada jurnal',
            'penulis': None, 'pembimbing_pertama': None, 'pembimbing_kedua': None,
            'abstrak': None, 'abstrak_inggris': None
        }]

    all_jurnal_data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        hasil_scrape = executor.map(scrape_jurnal_detail, jurnal_urls)
        for hasil in hasil_scrape:
            if hasil:
                hasil['id_prodi'] = prodi['id_prodi']
                hasil['nama_prodi'] = prodi['nama_prodi']
                hasil['nama_fakultas'] = prodi['nama_fakultas']
                all_jurnal_data.append(hasil)
    
    # Tambahkan \n di awal print agar lebih rapi
    print(f"\n✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Berhasil mengambil {len(all_jurnal_data)} jurnal.")
    return all_jurnal_data

def main():
    # ... (fungsi ini tidak berubah)
    start_time = time.time()
    daftar_prodi = get_fakultas_prodi_list()
    if not daftar_prodi: return pd.DataFrame()

    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results_per_prodi = executor.map(scrape_prodi, daftar_prodi)
        for list_jurnal in results_per_prodi:
            final_results.extend(list_jurnal)

    df = pd.DataFrame(final_results)
    if not df.empty:
        # Mengurutkan DataFrame berdasarkan urutan prodi asli
        original_prodi_order = [p['id_prodi'] for p in daftar_prodi]
        df['id_prodi'] = pd.Categorical(df['id_prodi'], categories=original_prodi_order, ordered=True)
        df = df.sort_values('id_prodi').reset_index(drop=True)
        
        df = df[['nama_fakultas', 'id_prodi', 'nama_prodi', 'judul', 'penulis', 'pembimbing_pertama', 'pembimbing_kedua', 'abstrak', 'abstrak_inggris']]
    
    df.to_csv("pta_final_concurrent_sorted.csv", index=False)
    
    end_time = time.time()
    print("\n\n✅ Proses scraping selesai.") # Tambah baris baru agar rapi
    print(f"Total baris data yang dihasilkan: {len(df)}.")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    return df

# if __name__ == "__main__":
#     df_hasil_cepat = main()
#     df_hasil_cepat

In [20]:
main()


✔️ Selesai: Hukum - Magister Ilmu Hukum | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Teknologi Industri Pertanian | Berhasil mengambil 4 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Ekonomi Pembangunan | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Agroteknologi | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Ilmu Kelautan | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Agribisnis | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Manajemen Sumberdaya Perairan | Berhasil mengambil 4 jurnal.

✔️ Selesai: Hukum - Ilmu Hukum | Berhasil mengambil 4 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Manajemen | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Magister Pengelolaan Sumber Daya Alam | Berhasil mengambil 3 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Doktor Ilmu Manajemen | Ditemukan 0 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Akuntansi | Berhasil mengambil 4 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - D3 Akuntansi | Berhasil mengambil 4 jurnal.

✔

,nama_fakultas,id_prodi,nama_prodi,judul,penulis,pembimbing_pertama,pembimbing_kedua,abstrak,abstrak_inggris
0,Hukum,1,Ilmu Hukum,Implementasi Fungsi Legislasi Dewan Perwakilan...,Dyah Ayu Citra Seza,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",ABSTRAK Implementasi Fungsi Legislasi DPRD Kab...,ABSTRACT Implementation of Legislation Parliam...
1,Hukum,1,Ilmu Hukum,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Maulina Nurlaily,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...
2,Hukum,1,Ilmu Hukum,Analisis Terhadap Kekosongan Hukum dalam Penga...,Moh. Samsul Hidayat,"Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi..."
3,Hukum,1,Ilmu Hukum,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,TOMMY ADITYA PARLINDUNGAN MARBUN,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...
4,Hukum,24,Magister Ilmu Hukum,PERTANGGUNG JAWABAN HUKUM PEJABAT PEMBUAT KOMI...,MOCHAMAD SAICHU,"Dr. Nunuk Nuswardani S.H., M.H","Dr. Deni SB Yuherawan S.H., Msi",Korupsi tergolong ke dalam kejahatan yang luar...,Korupsi tergolong ke dalam kejahatan yang luar...
...,...,...,...,...,...,...,...,...,...
137,Ilmu Pendidikan,30,Pgpaud,Pengaruh Strategi Meaningful Instructional Des...,Siti Herlinah Wifroh,"Yulias Wulani Fajar, M.Pd","Titin Faridatun Nisa', S.Pd., M.Pd",Tujuan penelitian ini untuk mengetahui pengaru...,The purpose of this study is to reveal the inf...
138,Ilmu Pendidikan,38,Pendidikan Profesi Guru,Tidak ada jurnal,None,None,None,None,None
139,Mbkm,99,Mbkm,Tidak ada jurnal,None,None,None,None,None
140,Pascasarjana,39,Magister Pendidikan Dasar,Tidak ada jurnal,None,None,None,None,None


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures

# MAKSIMAL PEKERJA SIMULTAN (BISA DISESUAIKAN)
MAX_WORKERS = 10

def get_fakultas_prodi_list():
    # Fungsi ini tidak berubah
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav: return []
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar prodi: {e}")
    return prodi_list

def scrape_jurnal_detail(jurnal_url):
    # Fungsi ini tidak berubah
    try:
        response = requests.get(jurnal_url, timeout=15)
        response.raise_for_status()
        isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
        if not isi: return None
        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
        abstract_paragraphs = isi.select('p[align="justify"]')
        text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else ""
        abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()
        text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else ""
        abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()
        return {
            "penulis": penulis, "judul": judul, "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua, "abstrak": abstrak_indonesia,
            "abstrak_inggris": abstrak_inggris
        }
    except requests.exceptions.RequestException:
        return None

# --- PERUBAHAN UTAMA ADA DI FUNGSI BERIKUT ---
def scrape_prodi(prodi):
    """
    Fungsi MANAJER: Sekarang mengambil SEMUA jurnal yang tersedia
    dan menampilkan ID prodi di log.
    """
    jurnal_urls = []
    page = 1
    
    # 1. MENGHAPUS BATAS 4 JURNAL
    # Loop akan berjalan terus sampai menemukan halaman kosong
    while True:
        try:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi['id_prodi']}/{page}"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals_on_page = soup.select('li[data-cat="#luxury"] a.gray.button')

            # Jika halaman kosong (tidak ada jurnal), hentikan pencarian untuk prodi ini
            if not jurnals_on_page:
                break
            
            # Ambil semua URL di halaman ini
            for a_tag in jurnals_on_page:
                jurnal_urls.append(a_tag['href'])
            
            page += 1
            time.sleep(0.5) # Beri jeda kecil antar halaman
        except requests.exceptions.RequestException:
            break # Hentikan jika ada error jaringan

    # Jika tidak ada URL jurnal yang ditemukan sama sekali
    if not jurnal_urls:
        # 2. MENAMBAHKAN ID PRODI PADA LOG
        print(f"\n✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} (ID: {prodi['id_prodi']}) | Ditemukan 0 jurnal.")
        return [{
            'id_prodi': prodi['id_prodi'], 'nama_prodi': prodi['nama_prodi'],
            'nama_fakultas': prodi['nama_fakultas'], 'judul': 'Tidak ada jurnal',
            'penulis': None, 'pembimbing_pertama': None, 'pembimbing_kedua': None,
            'abstrak': None, 'abstrak_inggris': None
        }]

    # Jika ada URL, proses scrape detail berjalan secara konkuren
    all_jurnal_data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        hasil_scrape = executor.map(scrape_jurnal_detail, jurnal_urls)
        for hasil in hasil_scrape:
            if hasil:
                hasil['id_prodi'] = prodi['id_prodi']
                hasil['nama_prodi'] = prodi['nama_prodi']
                hasil['nama_fakultas'] = prodi['nama_fakultas']
                all_jurnal_data.append(hasil)
    
    # 2. MENAMBAHKAN ID PRODI PADA LOG
    print(f"\n✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} (ID: {prodi['id_prodi']}) | Berhasil mengambil {len(all_jurnal_data)} dari {len(jurnal_urls)} jurnal yang ditemukan.")
    return all_jurnal_data

def main():
    # Fungsi ini tidak berubah
    start_time = time.time()
    daftar_prodi = get_fakultas_prodi_list()
    if not daftar_prodi: return pd.DataFrame()

    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results_per_prodi = executor.map(scrape_prodi, daftar_prodi)
        for list_jurnal in results_per_prodi:
            final_results.extend(list_jurnal)

    df = pd.DataFrame(final_results)
    if not df.empty:
        original_prodi_order = [p['id_prodi'] for p in daftar_prodi]
        df['id_prodi'] = pd.Categorical(df['id_prodi'], categories=original_prodi_order, ordered=True)
        df = df.sort_values('id_prodi').reset_index(drop=True)
        df = df[['nama_fakultas', 'id_prodi', 'nama_prodi', 'judul', 'penulis', 'pembimbing_pertama', 'pembimbing_kedua', 'abstrak', 'abstrak_inggris']]
    
    df.to_csv("pta_semua_jurnal_concurrent.csv", index=False)
    
    end_time = time.time()
    print("\n\n✅ Proses scraping selesai.")
    print(f"Total baris data yang dihasilkan: {len(df)}.")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    return df

if __name__ == "__main__":
    df_hasil_semua = main()
    df_hasil_semua